In [59]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [60]:
csvpath = '../Data/가구_Data/raw_Data/4.2.15_시·군·구별_근로소득_연말정산_신고현황_주소지_2016_20210912182408.csv'

tb_data1 = pd.read_csv(csvpath, encoding='cp949')

In [61]:
tb_data1.head(5)

,행정구역(시군구)별(1),행정구역(시군구)별(2),2016,2016.1,2016.2,2016.3,2016.4,2016.5,2016.6,2016.7,...,2018.6,2018.7,2019,2019.1,2019.2,2019.3,2019.4,2019.5,2019.6,2019.7
0,행정구역(시군구)별(1),행정구역(시군구)별(2),급여총계,급여총계,과세대상근로소득(총급여),과세대상근로소득(총급여),과세표준,과세표준,결정세액,결정세액,...,결정세액,결정세액,급여총계,급여총계,과세대상근로소득(총급여),과세대상근로소득(총급여),과세표준,과세표준,결정세액,결정세액
1,행정구역(시군구)별(1),행정구역(시군구)별(2),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),...,인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원)
2,전국,소계,17740098,600099695,17668285,595990661,15419516,320805519,9998156,30853854,...,11358784,38307766,19167273,721686447,19074871,717531015,17102113,398689340,12112622,41101084
3,서울,소계,3629625,138096837,3621150,137226200,3159795,79865568,2099483,10035736,...,2310586,12348196,3845841,164800618,3836212,163916825,3441286,98648176,2448702,13405669
4,서울,강남구,206668,13057288,206346,12989836,185324,9108357,139670,1761616,...,145315,2125841,209653,14926829,209360,14863887,191335,10618866,149796,2207827


In [62]:
tb_data1['행정구역'] = tb_data1['행정구역(시군구)별(1)'] + tb_data1['행정구역(시군구)별(2)']

tb_data1 = tb_data1[['행정구역', 
                     '2016', '2016.1', '2016.2', '2016.3', '2016.4', '2016.5', '2016.6', '2016.7', 
                     '2017', '2017.1', '2017.2', '2017.3', '2017.4', '2017.5', '2017.6', '2017.7', 
                     '2018', '2018.1', '2018.2', '2018.3', '2018.4', '2018.5', '2018.6', '2018.7', 
                     '2019', '2019.1', '2019.2', '2019.3', '2019.4', '2019.5', '2019.6', '2019.7',
                    ]]

In [63]:
tb_data1.head(5)

,행정구역,2016,2016.1,2016.2,2016.3,2016.4,2016.5,2016.6,2016.7,2017,...,2018.6,2018.7,2019,2019.1,2019.2,2019.3,2019.4,2019.5,2019.6,2019.7
0,행정구역(시군구)별(1)행정구역(시군구)별(2),급여총계,급여총계,과세대상근로소득(총급여),과세대상근로소득(총급여),과세표준,과세표준,결정세액,결정세액,급여총계,...,결정세액,결정세액,급여총계,급여총계,과세대상근로소득(총급여),과세대상근로소득(총급여),과세표준,과세표준,결정세액,결정세액
1,행정구역(시군구)별(1)행정구역(시군구)별(2),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),...,인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원)
2,전국소계,17740098,600099695,17668285,595990661,15419516,320805519,9998156,30853854,18005534,...,11358784,38307766,19167273,721686447,19074871,717531015,17102113,398689340,12112622,41101084
3,서울소계,3629625,138096837,3621150,137226200,3159795,79865568,2099483,10035736,3645417,...,2310586,12348196,3845841,164800618,3836212,163916825,3441286,98648176,2448702,13405669
4,서울강남구,206668,13057288,206346,12989836,185324,9108357,139670,1761616,201206,...,145315,2125841,209653,14926829,209360,14863887,191335,10618866,149796,2207827


In [64]:
type(tb_data1)
tb_data1.shape

ls = tb_data1.columns + tb_data1.iloc[0] + tb_data1.iloc[1]
new_namedic = {}
for i in range(len(tb_data1.columns)): 
    
    new_name = tb_data1.columns[i].split('.')[0] + " " + tb_data1.iloc[0][i] + " " + tb_data1.iloc[1][i] 
    if i == 0:
        new_name = tb_data1.columns[i]
    new_namedic[tb_data1.columns[i]] = new_name
    
tb_data1.rename(columns=new_namedic, inplace=True)



In [65]:
tb_data1

,행정구역,2016 급여총계 인원 (명),2016 급여총계 금액 (백만원),2016 과세대상근로소득(총급여) 인원 (명),2016 과세대상근로소득(총급여) 금액 (백만원),2016 과세표준 인원 (명),2016 과세표준 금액 (백만원),2016 결정세액 인원 (명),2016 결정세액 금액 (백만원),2017 급여총계 인원 (명),...,2018 결정세액 인원 (명),2018 결정세액 금액 (백만원),2019 급여총계 인원 (명),2019 급여총계 금액 (백만원),2019 과세대상근로소득(총급여) 인원 (명),2019 과세대상근로소득(총급여) 금액 (백만원),2019 과세표준 인원 (명),2019 과세표준 금액 (백만원),2019 결정세액 인원 (명),2019 결정세액 금액 (백만원)
0,행정구역(시군구)별(1)행정구역(시군구)별(2),급여총계,급여총계,과세대상근로소득(총급여),과세대상근로소득(총급여),과세표준,과세표준,결정세액,결정세액,급여총계,...,결정세액,결정세액,급여총계,급여총계,과세대상근로소득(총급여),과세대상근로소득(총급여),과세표준,과세표준,결정세액,결정세액
1,행정구역(시군구)별(1)행정구역(시군구)별(2),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),...,인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원),인원 (명),금액 (백만원)
2,전국소계,17740098,600099695,17668285,595990661,15419516,320805519,9998156,30853854,18005534,...,11358784,38307766,19167273,721686447,19074871,717531015,17102113,398689340,12112622,41101084
3,서울소계,3629625,138096837,3621150,137226200,3159795,79865568,2099483,10035736,3645417,...,2310586,12348196,3845841,164800618,3836212,163916825,3441286,98648176,2448702,13405669
4,서울강남구,206668,13057288,206346,12989836,185324,9108357,139670,1761616,201206,...,145315,2125841,209653,14926829,209360,14863887,191335,10618866,149796,2207827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,경남합천군,9315,262452,9204,260814,8048,127450,4919,7764,9187,...,5545,9386,9211,290330,9064,288954,8132,147990,5706,9948
245,제주소계,197972,5705113,197102,5674147,169719,2826393,101628,226606,207528,...,126097,288522,225998,7297193,225382,7266684,200482,3748591,136871,314130
246,제주제주시,150621,4389021,150155,4366483,129793,2174513,77749,171441,157011,...,95694,218142,171469,5584793,171144,5562873,152621,2862584,104244,235874
247,제주서귀포시,47351,1316092,46947,1307664,39926,651880,23879,55165,50517,...,30403,70380,54529,1712400,54238,1703811,47861,886007,32627,78256


In [66]:
tb_data1.drop(0, inplace = True)
tb_data1.drop(1, inplace = True)

In [67]:
tb_data1.reset_index(drop=True, inplace=True)

In [68]:
tb_data1

,행정구역,2016 급여총계 인원 (명),2016 급여총계 금액 (백만원),2016 과세대상근로소득(총급여) 인원 (명),2016 과세대상근로소득(총급여) 금액 (백만원),2016 과세표준 인원 (명),2016 과세표준 금액 (백만원),2016 결정세액 인원 (명),2016 결정세액 금액 (백만원),2017 급여총계 인원 (명),...,2018 결정세액 인원 (명),2018 결정세액 금액 (백만원),2019 급여총계 인원 (명),2019 급여총계 금액 (백만원),2019 과세대상근로소득(총급여) 인원 (명),2019 과세대상근로소득(총급여) 금액 (백만원),2019 과세표준 인원 (명),2019 과세표준 금액 (백만원),2019 결정세액 인원 (명),2019 결정세액 금액 (백만원)
0,전국소계,17740098,600099695,17668285,595990661,15419516,320805519,9998156,30853854,18005534,...,11358784,38307766,19167273,721686447,19074871,717531015,17102113,398689340,12112622,41101084
1,서울소계,3629625,138096837,3621150,137226200,3159795,79865568,2099483,10035736,3645417,...,2310586,12348196,3845841,164800618,3836212,163916825,3441286,98648176,2448702,13405669
2,서울강남구,206668,13057288,206346,12989836,185324,9108357,139670,1761616,201206,...,145315,2125841,209653,14926829,209360,14863887,191335,10618866,149796,2207827
3,서울강동구,161837,5614760,161558,5577333,140810,3045249,91771,298009,160371,...,101758,370841,183833,7447177,183504,7407086,164583,4268152,118169,469757
4,서울강북구,102249,2647871,102053,2632574,86391,1248641,50278,84141,102117,...,56233,99598,105025,3075447,104773,3058999,91856,1525316,59757,109023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,경남합천군,9315,262452,9204,260814,8048,127450,4919,7764,9187,...,5545,9386,9211,290330,9064,288954,8132,147990,5706,9948
243,제주소계,197972,5705113,197102,5674147,169719,2826393,101628,226606,207528,...,126097,288522,225998,7297193,225382,7266684,200482,3748591,136871,314130
244,제주제주시,150621,4389021,150155,4366483,129793,2174513,77749,171441,157011,...,95694,218142,171469,5584793,171144,5562873,152621,2862584,104244,235874
245,제주서귀포시,47351,1316092,46947,1307664,39926,651880,23879,55165,50517,...,30403,70380,54529,1712400,54238,1703811,47861,886007,32627,78256


In [69]:
tb_data1.columns

for col in tb_data1.columns:
    print(col)
    if ("과세표준" in col) or ("결정세액" in col) :
        tb_data1.drop(col, axis=1, inplace=True)
        print("deleted")

행정구역
2016 급여총계 인원 (명)
2016 급여총계 금액 (백만원)
2016 과세대상근로소득(총급여) 인원 (명)
2016 과세대상근로소득(총급여) 금액 (백만원)
2016 과세표준 인원 (명)
deleted
2016 과세표준 금액 (백만원)
deleted
2016 결정세액 인원 (명)
deleted
2016 결정세액 금액 (백만원)
deleted
2017 급여총계 인원 (명)
2017 급여총계 금액 (백만원)
2017 과세대상근로소득(총급여) 인원 (명)
2017 과세대상근로소득(총급여) 금액 (백만원)
2017 과세표준 인원 (명)
deleted
2017 과세표준 금액 (백만원)
deleted
2017 결정세액 인원 (명)
deleted
2017 결정세액 금액 (백만원)
deleted
2018 급여총계 인원 (명)
2018 급여총계 금액 (백만원)
2018 과세대상근로소득(총급여) 인원 (명)
2018 과세대상근로소득(총급여) 금액 (백만원)
2018 과세표준 인원 (명)
deleted
2018 과세표준 금액 (백만원)
deleted
2018 결정세액 인원 (명)
deleted
2018 결정세액 금액 (백만원)
deleted
2019 급여총계 인원 (명)
2019 급여총계 금액 (백만원)
2019 과세대상근로소득(총급여) 인원 (명)
2019 과세대상근로소득(총급여) 금액 (백만원)
2019 과세표준 인원 (명)
deleted
2019 과세표준 금액 (백만원)
deleted
2019 결정세액 인원 (명)
deleted
2019 결정세액 금액 (백만원)
deleted


In [70]:
tb_data1

,행정구역,2016 급여총계 인원 (명),2016 급여총계 금액 (백만원),2016 과세대상근로소득(총급여) 인원 (명),2016 과세대상근로소득(총급여) 금액 (백만원),2017 급여총계 인원 (명),2017 급여총계 금액 (백만원),2017 과세대상근로소득(총급여) 인원 (명),2017 과세대상근로소득(총급여) 금액 (백만원),2018 급여총계 인원 (명),2018 급여총계 금액 (백만원),2018 과세대상근로소득(총급여) 인원 (명),2018 과세대상근로소득(총급여) 금액 (백만원),2019 급여총계 인원 (명),2019 급여총계 금액 (백만원),2019 과세대상근로소득(총급여) 인원 (명),2019 과세대상근로소득(총급여) 금액 (백만원)
0,전국소계,17740098,600099695,17668285,595990661,18005534,637616487,17925345,633611675,18577885,681611264,18501089,677488617,19167273,721686447,19074871,717531015
1,서울소계,3629625,138096837,3621150,137226200,3645417,146379773,3636049,145511903,3743474,155269460,3734108,154395729,3845841,164800618,3836212,163916825
2,서울강남구,206668,13057288,206346,12989836,201206,14142558,200894,14078169,206821,14403588,206523,14339618,209653,14926829,209360,14863887
3,서울강동구,161837,5614760,161558,5577333,160371,5824940,160084,5788728,165248,6291291,164967,6254566,183833,7447177,183504,7407086
4,서울강북구,102249,2647871,102053,2632574,102117,2764297,101879,2749477,103391,2913526,103160,2897863,105025,3075447,104773,3058999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,경남합천군,9315,262452,9204,260814,9187,270618,9044,269179,9175,283536,9148,282176,9211,290330,9064,288954
243,제주소계,197972,5705113,197102,5674147,207528,6282535,206746,6252334,217451,6820187,216873,6790174,225998,7297193,225382,7266684
244,제주제주시,150621,4389021,150155,4366483,157011,4803882,156638,4781549,164636,5213420,164312,5191500,171469,5584793,171144,5562873
245,제주서귀포시,47351,1316092,46947,1307664,50517,1478653,50108,1470785,52815,1606768,52561,1598675,54529,1712400,54238,1703811


In [71]:
tb_data1.to_csv("../Data/가구_Data/연말정산 근로 소득 신고 인원 및 금액(16-19).csv")